# This note book shows how to load header file and waveform data and plot it.

In [ ]:
import pandas as pd
from datetime import datetime
import os
import pickle
from datetime import timedelta
from datetime import datetime
from datetime import date
from datetime import time
from dateutil.relativedelta import relativedelta

import pandas as pd
import numpy as np
import os

import json

import sys
from scipy.io import loadmat

import matplotlib
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
matplotlib.rc( 'savefig', facecolor = 'white' )
from matplotlib import pyplot
import matplotlib.ticker as plticker

import seaborn as sns


import sys
sys.path.append('../') # add this line so Data and data are visible in this file
sys.path.append('../../') # add this line so Data and data are visible in this file
sys.path.append('../PhysioMC/') # add this line so Data and data are visible in this file

# from PatchWand import *
from filters import *
from setting import *
# from preprocessing import *
from ECG_module import *
from dataIO import *
from stage1_PPG_analysis import *

from importlib import reload
%load_ext autoreload
%autoreload 2

In [ ]:
# %pip install feather-format

In [ ]:
# %pip install parfor

In [ ]:
selected_id = 5

In [ ]:
%pwd

# load csv files David prepared

## load the demographic data, only look at black or white subjects for now

In [ ]:
demographic_filedir = '/home/mchan82/kamaleswaranlab/Sepsis3_062321/dlin_analysis/Pat_BedTime_List_some_on_BMI_2_6_22_dem.csv'
df_demographic = pd.read_csv(demographic_filedir, index_col=0)  

# mask out subjects who are not black and white
mask = (df_demographic['Race String']=='African American  or Black') | (df_demographic['Race String']=='Caucasian or White')

print('only {:.2f}% is black or white'.format(mask.mean()*100))
df_demographic = df_demographic[mask]

In [ ]:
df_demographic

## load the oxygen saturation data
### TODO: need to add FiO2 and PaO2 data

In [ ]:
# FiO2 and PaO2? (PaO2 / FiO2 < 400 indicates respiratory problem)
oxygen_sat_filedir = '/home/mchan82/kamaleswaranlab/Sepsis3_062321/dlin_analysis/Pat_BedTime_List_some_on_BMI_2_6_22_spo2_sao2.csv'
df_oxygen_sat = pd.read_csv(oxygen_sat_filedir, index_col=0)  

In [ ]:
mask = (df_oxygen_sat['Black']==1) | (df_oxygen_sat['White']==1)
df_oxygen_sat = df_oxygen_sat[mask]

In [ ]:
df_oxygen_sat['times'] =  pd.to_datetime(df_oxygen_sat['times'], format="%Y-%m-%d %H:%M:%S")


In [ ]:
df_oxygen_sat = df_oxygen_sat.rename(columns = {'pat_id':'PAT_ID', 'csn':'CSN'})
df_oxygen_sat

In [ ]:
df_oxygen_sat.describe()

In [ ]:
df_oxygen_sat['CSN'].unique().shape

## load the ind csv
### compute BED Time Diff (to show that we have much more bed data)
### show the median the time diffs

In [ ]:
ind_filedir = '/home/mchan82/kamaleswaranlab/Sepsis3_062321/dlin_analysis/Pat_BedTime_List_some_on_BMI_2_6_22_ind.csv'
df_ind = pd.read_csv(ind_filedir, index_col=0)  

mask = (df_ind['Race String']=='African American  or Black') | (df_ind['Race String']=='Caucasian or White')
df_ind = df_ind[mask]

In [ ]:
df_ind['DateStart'] = pd.to_datetime(df_ind['DateStart'], format="%Y-%m-%d %H:%M:%S")
df_ind['DateEnd'] = pd.to_datetime(df_ind['DateEnd'], format="%Y-%m-%d %H:%M:%S")

## TODO: change BED Time Difference

In [ ]:
BED_LOCATION_START = pd.to_datetime(df_ind['BED_LOCATION_START'], format="%Y-%m-%d %H:%M:%S")
BED_LOCATION_END = pd.to_datetime(df_ind['BED_LOCATION_END'], format="%m/%d/%Y %H:%M:%S")

df_ind['BED_LOCATION_START'] = BED_LOCATION_START
df_ind['BED_LOCATION_END'] = BED_LOCATION_END


In [ ]:
DateStart = pd.to_datetime(df_ind['DateStart'], format="%Y-%m-%d %H:%M:%S")
DateEnd = pd.to_datetime(df_ind['DateEnd'], format="%m/%d/%Y %H:%M:%S")

df_ind['DateStart'] = DateStart
df_ind['DateEnd'] = DateEnd

df_ind['BED Time Difference'] = (DateEnd-DateStart).dt.total_seconds()

In [ ]:
print('EHR time diff: {:.2f}hr'.format(df_ind['Time Difference'].median()/60/60))
print('Waveform time diff: {:.2f}hr'.format(df_ind['BED Time Difference'].median()/60/60))

In [ ]:
df_ind

In [ ]:
df_ind['CSN'].unique().shape

In [ ]:
df_ind['Time Difference']

# merge df_ind with df_oxygen_sat so I can get deBedTime, DateStart, DateEnd, PAT_ID, White	Black, CSN, SaO2, SpO2, oxy sat. times in one column

## TODO: merge using the CSN instead 

In [ ]:
df_merged = df_ind.merge(df_oxygen_sat[['spo2', 'sao2', 'times', 'PAT_ID', 'CSN']], on=['CSN', 'PAT_ID'], how='outer')
df_merged

In [ ]:
(df_merged['DateEnd']=='1969-12-31 23:59:58').mean()

In [ ]:
(df_merged['DateEnd']>'1970-12-31 23:59:58').mean()

In [ ]:
# df_merged[(df_merged['PAT_ID']==PAT_ID) & (df_merged['deBedTime']=='A140-0515603029')]

# But simply merging them using PAT_ID is not ideal: same PAT_ID may have several deBedTime
## need to filter out rows incorrectly align by checking the timing
## only associate the demographic data to the ox sat. data when the timing match

In [ ]:
DateStart = pd.to_datetime(df_merged['DateStart'], format="%Y-%m-%d %H:%M:%S") # str to datetime
DateEnd = pd.to_datetime(df_merged['DateEnd'], format="%Y-%m-%d %H:%M:%S")
times = pd.to_datetime(df_merged['times'], format="%Y-%m-%d %H:%M:%S")

In [ ]:
# timedelta: int to datetime
mask = (times-timedelta(minutes=min_threshold) >= DateStart) & (times+timedelta(minutes=min_threshold) <= DateEnd)
df_merged = df_merged[mask]

In [ ]:
print(mask.mean())

In [ ]:
df_merged['CSN'].unique().shape

In [ ]:
df_merged['PAT_ID'].unique().shape

In [ ]:
df_merged = df_merged.reset_index(drop=True)
df_merged

In [ ]:
# (24*60*60)/149062

# get PPG data

## os.listdir(inputdir) = 
## ['A071-0512777549',
## 'A063-0482186017',
## 'A113-0520303111', ...]

In [ ]:
# waveform_window

## os.listdir(inputdir_rec) = 
## ['A058-0502749430_0028d.mat',
##  'A058-0502749430.almth',
##  'A058-0502749430_0013.mat',
##  'A058-0502749430_0013.hea',...]

In [ ]:
outputdir = '../../data/stage1/'
if not os.path.exists(outputdir):
    os.makedirs(outputdir)

In [ ]:
with open('recording_lookup.json') as json_file:
    recording_lookup = json.load(json_file)


In [ ]:
homedir = os.path.expanduser('~')+'/'

# this is where all waveform data are stored
inputdir = '/labs/kamaleswaranlab/Sepsis3_062321/Files_cp_sftp_6_30_21/Files_cp_6_30_21/'

# yes
# rec_id = 'A058-0502749430'

# no
# rec_id = 'A030-0497283634'
# rec_id = 'A097-0427935662'
# rec_id = 'A085-0500344504'


# # check the header files in here
# for rec_file in os.listdir(inputdir+rec_id):
#     if 'hea' in rec_file:
#         print(rec_file)

In [ ]:
# TODO: move to dataIO.py, modularize it
def get_header_valid_waveform(inputdir_rec, row):

    # header_list = []
    for header_name in sorted(os.listdir(inputdir_rec)):
        
        if '.hea' not in header_name:
            continue
        if '_' not in header_name:
            continue
        if '_0n' in header_name:
            continue
        if '_1n' in header_name:
            continue
        if 'layout' in header_name:
            continue

        # print(header_name)
        headerdir = inputdir_rec + header_name


        header_id = header_name.split('_')[-1].split('.hea')[0]
        # print(header_id)
        # print(int(header_id))
        # sys.exit()

        with open(headerdir, 'r') as f:
            header = f.readlines()

        header_dict = header_extract(header)

        # print(header_name)

        allheader_hours = int(header_id)*8
        currentheader_hours = header_dict['t_dur(hr)']

        # waveform_start = row['BED_LOCATION_START'] + timedelta(hours=allheader_hours)
        # waveform_end = row['BED_LOCATION_START'] + timedelta(hours=allheader_hours) + timedelta(hours=currentheader_hours)

        # using DateStart since it is the start time of the waveform
        waveform_start = row['DateStart'] + timedelta(hours=allheader_hours)
        waveform_end = row['DateStart'] + timedelta(hours=allheader_hours) + timedelta(hours=currentheader_hours)

        header_dict['waveform_start'] = waveform_start
        header_dict['waveform_end'] = waveform_end
        header_dict['header_name'] = header_name
        header_dict['header_id'] = header_id

        # header_list.append(header_dict)
        
    # return header_list
        if (waveform_start > row['times']) or (waveform_end < row['times']):
            # print('wrong recording')
            continue
        else:
            # print(header_start, header_end, row['times'])
            # print('bingo!')
            return header_dict


        return None
        # print('\t', header_dict['start_time'], header_dict['date'])

    # header_id = header_name.split('_')[-1]

In [ ]:
outputdir_recording = outputdir+'waveform/'
if not os.path.exists(outputdir_recording):
    os.makedirs(outputdir_recording)
    


In [ ]:
# df_demographic_processed = pd.read_csv(outputdir_recording+'df_demographic_processed.csv.gz').sort_values('index').reset_index(drop=True)


In [ ]:
# df_demographic_processed.shape[0]/df_merged.shape[0]

# TODO
## 1. filter the signal first (DONE)
## 2. apply QRS detector and segment the beats, fix the beat plotting function below (may not need this anymore, check get_sig2beats)
## 3. store the PPG (and ECG?) beats in npz (2D matrix) (DONE)
## 4. appned a column to df_merged that point to the matrix (file name) (DONE)
## 5. parallel computing (DONE)

In [ ]:
# df_merged.shape

In [ ]:
def process_row(row, index, inputdir, outputdir_recording):
    



    # 1. get header_dict
    rec_id = row['deBedTime']
    debug_message = '[perfecto]'

    print('\n==================  working on rec_id: {} =================='.format(rec_id) )
    inputdir_rec = inputdir + rec_id + '/'

    
    # print(os.listdir(inputdir_rec))
    


    # continue
    # header_dict = get_header_valid(inputdir_rec) # None or some dictionary data
    header_dict = get_header_valid_waveform(inputdir_rec, row) # None or some dictionary data


    # print(header_dict)

    # 2. continue if waveform is not within the header_dict timing range
    if header_dict is None:
        debug_message = '[DNE] wrong recording'
        if verbose:
            print(debug_message)
            print('\t', row['deBedTime'], row['spo2'], row['sao2'], row['times'], row['Black'], row['White'])
    
    else:
        if 'SPO2' not in list(header_dict['unit_dict'].keys()):
            debug_message = '[DNE] SPO2 channel not in waveform'
            if verbose:
                print(debug_message)

    if 'perfecto' in debug_message:

        # 3. get the waveform data
        df_bed, _ = get_df_bed(rec_id, inputdir, header_dict['header_id']) # RX limiting factor
        df_bed = df_bed[['I', 'SPO2', 'time']]

        

        
        
        # 4. get only a small piece of the waveform data
        sync_start = row['times'] - timedelta(minutes=waveform_window)
        sync_end = row['times'] + timedelta(minutes=waveform_window)

        i_sync_start = int((sync_start - header_dict['waveform_start']).total_seconds() * header_dict['Fs'])
        i_sync_end = int((sync_end - header_dict['waveform_start']).total_seconds() * header_dict['Fs'])

        df_bed = df_bed[i_sync_start:i_sync_end]


    
        # 5. check on the waveform data. Continue if the signals are off
        if df_bed.shape[0] < waveform_window*60*header_dict['Fs']/2:
            debug_message = '[Inadequate] data size smaller than {:.1f}s'.format( waveform_window/2 )
            if verbose:
                print(debug_message)
                print('\t', header_dict['header_name'], header_dict['waveform_start'],header_dict['waveform_end'],header_dict['t_dur(hr)'])
                print('\t', row['DateStart'], row['DateEnd'])
                print('\t', df_bed['time'].max()-df_bed['time'].min() )



    
    if 'perfecto' in debug_message:
        # get the waveform, segment it, store it     
        PPGmatrix, ECGmatrix, debug_message = df_bed2PPGmatrix(df_bed) # df_bed will be resampled to FS_RESAMPLE
        # print(PPGmatrix.shape, ECGmatrix.shape)
        # sys.exit()
        
        # select one of the window randomly
        PPGmatrix = PPGmatrix[:,np.random.randint(PPGmatrix.shape[1])]
        ECGmatrix = ECGmatrix[:,np.random.randint(ECGmatrix.shape[1])]
        # print(PPGmatrix.shape)

        if 'perfecto' not in debug_message:
            PPGmatrix_name = 'DNE'
            ECGmatrix_name = 'DNE'
        else:
            PPGmatrix_name = 'PPGmatrix_{}_{}_{}'.format(rec_id, header_dict['header_id'], index)  
            data_saver(PPGmatrix, PPGmatrix_name, outputdir_recording)
            
            ECGmatrix_name = 'ECGmatrix_{}_{}_{}'.format(rec_id, header_dict['header_id'], index)  
            data_saver(ECGmatrix, ECGmatrix_name, outputdir_recording)
            # data_saver(PPGmatrix, 'data', outputdir_recording)
        # PPGmatrix = data_loader('sig_segment_A035-0519626285_0000_7581', '../../data/stage1/waveform/')
        # row = pd.concat([row, pd.Series({'matrix_dir': matrix_dir, 'index': index, 'debug_message': debug_message})])

    # if 'perfecto' in debug_message:
        # row = pd.concat([row, pd.Series({'matrix_dir': matrix_dir, 'index': index, 'debug_message': debug_message})])
    else:
        PPGmatrix_name = 'DNE'
        ECGmatrix_name = 'DNE'
        
    row = pd.concat([row, pd.Series({'PPGmatrix_name': PPGmatrix_name, 'ECGmatrix_name': ECGmatrix_name, 'index': index, 'debug_message': debug_message})])

    row =  pd.DataFrame(row).T
    row.to_feather(outputdir_recording+'df_demographic_processed_{}.feather'.format(index))

    
    return row
    # df_demographic_processed = pd.concat([df_demographic_processed, pd.DataFrame(row).T])
    # df_demographic_processed.reset_index(drop=True).to_csv(outputdir_recording+'df_demographic_processed.csv.gz', index=False, compression='gzip')


In [ ]:
df_merged

In [ ]:
# df_demographic_processed = pd.read_csv(outputdir_recording+'df_demographic_processed.csv.gz').sort_values('index').reset_index(drop=True)
# df_demographic_processed.shape

# loop for parsing the rows (don't delete)

In [ ]:
# if ~from_scratch:
#     print('hi')

In [ ]:
# ~from_scratch

In [ ]:
files_in_outputdir_recording = os.listdir(outputdir_recording)

In [ ]:
files_in_outputdir_recording[0]

In [48]:
waveform_window = 0.5 # minute,  preceeding and proceeding
verbose=False
from_scratch = False

if from_scratch:
    df_demographic_processed = pd.DataFrame()
else:
    # df_demographic_processed = pd.read_csv(outputdir_recording+'df_demographic_processed.csv.gz').sort_values('index').reset_index(drop=True)
    df_demographic_processed = pd.read_feather(outputdir_recording+'df_demographic_processed.feather', columns=None, use_threads=True)

    print(df_demographic_processed.shape)
# df_demographic_processed



for index, row in df_merged.iterrows():
    # start = time.time()
    
    if not from_scratch: # if not from scatch, check if index is in df_demographic_processed['index']
        if index in df_demographic_processed['index']:
            # print(index)
            continue
            
        if 'df_demographic_processed_{}.feather'.format(index) in files_in_outputdir_recording:
            continue
            
            if os.path.getsize(outputdir_recording+'df_demographic_processed_{}.feather'.format(index))==0:
                continue
    
    row_processed = process_row(row, index, inputdir, outputdir_recording)
    
    
    # df = pd.read_feather(outputdir_recording+'df_demographic_processed_{}.feather'.format(index), columns=None, use_threads=True)

    # sys.exit()
    
    df_demographic_processed = pd.concat([row_processed, df_demographic_processed])
    # df_demographic_processed.reset_index(drop=True).to_csv(outputdir_recording+'df_demographic_processed.csv.gz', index=False, compression='gzip')
    df_demographic_processed.reset_index(drop=True).to_feather(outputdir_recording+'df_demographic_processed.feather')

    
    
    
    
    
    # sys.exit()

    # continue

    # end = time.time()
    # print(end - start)

ArrowInvalid: Not an Arrow file

In [ ]:
# ==================  working on rec_id: A088-0516752917 ==================
# row_processed

In [ ]:
df_demographic_processed

In [ ]:
sys.exit()

# parallel computing

In [ ]:
from joblib import Parallel, delayed
import multiprocessing

import time

start = time.time()

df_demographic_processed = pd.DataFrame()

num_cores = multiprocessing.cpu_count()

results = Parallel(n_jobs=num_cores)(delayed(process_row)(row, index, inputdir, outputdir_recording) for index, row in df_merged.iterrows())
df_demographic_processed = pd.concat(results)
df_demographic_processed.reset_index(drop=True).to_csv(outputdir_recording+'df_demographic_processed.csv.gz', index=False, compression='gzip')

end = time.time()
print(end - start)

In [ ]:
df = pd.read_csv(outputdir_recording+'df_demographic_processed.csv.gz')

In [ ]:
# results

In [ ]:
sys.exit()

# test on one random file

# get the header file

In [ ]:
rec_id = list(recording_lookup.keys())[selected_id]

bed_id = rec_id.split('-')[0]
subject_id = rec_id.split('-')[1]

header_id = recording_lookup[rec_id]
header_name = rec_id+'_{}.hea'.format(header_id)

# get recording directory
inputdir_rec = inputdir + rec_id + '/'

# get header file directory
headerdir = inputdir_rec + header_name
print(headerdir)

# load header file
with open(headerdir, 'r') as f:
    header = f.readlines()

header_dict = header_extract(header)

In [ ]:
header_dict['header_id'] = header_id

In [ ]:
header_dict

In [ ]:
# recording_lookup

# get raw waveform data

In [ ]:
df_bed, _ = get_df_bed(rec_id, inputdir, header_dict['header_id'])
df_bed

# duration of data

In [ ]:
t_arr = np.arange(df_bed.shape[0])/header_dict['Fs']

df_bed['time'] = t_arr
df_bed['time'] = df_bed['time']-df_bed['time'].values[0]

t_dur = df_bed.shape[0]/header_dict['Fs']
print('t_dur: {:.2f}s'.format(t_dur))

In [ ]:
header_dict['unit_dict']

In [ ]:
plot_all_sync(df_bed, rec_id, header_dict, t_start=300, t_end=400, plt_scale=0.3, fig_name=None, outputdir=None, show_plot=True)


In [ ]:
# extract beat matrix

In [ ]:
seg_duration = 30 * 60 # sec

Fs = header_dict['Fs']
N_segs = int((df_bed.shape[0]/Fs)//seg_duration)
verbose = True

df_features_bed = pd.DataFrame()

for i_seg in range(N_segs):

    if i_seg>3:
        continue

    if verbose:
        print('i_seg: ', i_seg)

    i_start = int(i_seg*seg_duration*Fs)
    i_end = int((i_seg+1)*seg_duration*Fs)

    df_seg = df_bed[i_start:i_end].copy()

    if df_seg.shape[0]==0:
        continue
    if np.mean(np.diff(df_seg['SPO2'])==0)>signal_quite_threshold:
        continue
    if np.mean(np.diff(df_seg['I'])==0)>signal_quite_threshold:
        continue
    if np.mean(np.diff(df_seg['II'])==0)>signal_quite_threshold:
        continue
    if np.mean(np.diff(df_seg['III'])==0)>signal_quite_threshold:
        continue
    if np.mean(np.diff(df_seg['V'])==0)>signal_quite_threshold:
        continue


    norm1 = np.abs(get_ecg_IP_norm(df_seg['I'].values, df_seg['II']))
    norm2 = np.abs(get_ecg_IP_norm(df_seg['I'].values, df_seg['III']))
    norm3 = np.abs(get_ecg_IP_norm(df_seg['I'].values, df_seg['V']))

    if (norm1 < 0.3) or (norm2 < 0.3) or (norm3 < 0.3):
        if verbose:
            print('\tbad ECG')
        continue


    sig_name = 'I'
    ECG = df_seg[sig_name].values


    # ECG = df[sig_name].values

    fig_name = 'ECG_diagnostics_'+sig_name
    # QRS_detector_dict = task_HR_detector(ECG, Fs, fig_name=fig_name, outputdir=None, show_plot=True)
    QRS_detector_dict = task_HR_detector(ECG, Fs, fig_name=fig_name, outputdir=None, show_plot=False)

#     t_ecg = np.arange(ECG.shape[0])/Fs
#     hr_interp = np.interp(t_ecg, QRS_detector_dict['ts_hr'], QRS_detector_dict['hr'])


    # segment the beats
    beats_dict = segment_df(df_seg, QRS_detector_dict, Fs)
    # mask_ppg, ol_rate = clean_PPG(beats_dict['SPO2'], beats_dict['SPO2'].mean(axis=1), Fs)
    # beats_dict = masking_beats_dict(beats_dict, mask_ppg)

    sys.exit()

In [ ]:
# TODO: check segment_df, validate the beat plotting function again

In [ ]:
get_filt_df

In [ ]:
# plt.plot(beats_dict['I'][:50, :], color='gray', alpha=0.5)
# plt.show()

In [ ]:
# beats_dict['I'].shape

In [ ]:
plt.plot(beats_dict['SPO2'][:50, :], color='gray', alpha=0.1)
plt.show()

In [ ]:
# # create a list
# prime_numbers = [2, 3, 5, 7, 9, 11]

# # remove 9 from the list
# prime_numbers.remove(1)

In [ ]:
# prime_numbers

In [ ]:
plot_all_sync(df_seg, rec_id, header_dict, t_start=300, t_end=400, plt_scale=0.3, fig_name=None, outputdir=None, show_plot=True)


# # TODO: need to renovate this function so it can work again


In [ ]:
def plot_ALL_beats2(beats_dict, beats_id, subject_id, Fs, show_good=None, fig_name=None, outputdir=None, show_plot=False):
    beats_id = [1,2,3,4,5,6, 7]
    beats_names = ['I', 'II', 'III', 'V', 'SPO2', 'PPG_DC', 'AR2']
    unit_dict = header_dict['unit_dict']
    unit_dict['PPG_DC'] = ''
    t_beat = np.arange(beats_dict['I'].shape[0])/Fs

    fig = plt.figure(figsize=(16, 10), dpi=80)
    fontsize = 20
    alpha = 0.03


    # for (beat_name, beat_i) in zip(beats_dict, beats_id):
    for (beat_name, beat_i) in zip(beats_names, beats_id):

        beats = beats_dict[beat_name]

        ax = fig.add_subplot(3, 4, beat_i)
        ax.set_title(beat_name+'\n', fontsize=fontsize)

        show_good = True
        if show_good is not None:


            template = beats_dict[beat_name].mean(axis=1)
            if beat_name=='SPO2':
                mask_all, ol_rate = clean_PPG(beats, template, Fs)
                ax.set_title(beat_name+'\nacception_rate:{:.2f}'.format(1-ol_rate), fontsize=fontsize)


        ax.plot(t_beat, beats, color='gray', alpha=alpha)
        ax.plot(t_beat, np.mean(beats,axis=1), color='firebrick', linewidth=3)

        if 'SPO2' in beat_name or 'I' == beat_name or 'II' == beat_name or 'III' == beat_name or 'V' == beat_name:
            beats_mean = np.mean(beats,axis=1)

            ymin = beats_mean.mean() - beats_mean.std()*5
            ymax = beats_mean.mean() + beats_mean.std()*5
            ax.set_ylim(ymin, ymax)

        ax.tick_params(axis='both', which='major', labelsize=13)
        ax.set_ylabel(unit_dict[beat_name], fontsize=fontsize-3)
        ax.set_xlabel('time (sec)', fontsize=fontsize)



    fig.tight_layout()

    # if outputdir is not None:

    #     if fig_name is None:
    #         fig_name = 'beats_ensemble_sub{}'.format(subject_id)

    #     if not os.path.exists(outputdir):
    #         os.makedirs(outputdir)
    #     fig.savefig(outputdir + fig_name+'.png', transparent=False)

    # if show_plot == False:
    #     plt.close(fig)
    #     pyplot.close(fig)
    #     plt.close('all')